In [25]:
from __future__ import annotations

import os
import sys
import time
import psutil
import timeit
from typing import Literal
from copy import deepcopy

from multiprocessing.pool import ThreadPool

In [3]:
def _build_arr(length: int, fill = None) -> list:
    return [fill] * length

def _add_arr(arr1, arr2) -> list[float]:
    return [v + arr2[i] for i, v in enumerate(arr1)]

In [19]:
class Matrix:
    _cpu_count = psutil.cpu_count()
    print(_cpu_count)

    def __init__(self, data: list[list[int]], copy: bool = True) -> None:
        self._data = deepcopy(data) if copy else data
        self._shape = (len(data), len(data[0]))
        self._size = self._shape[0] * self._shape[1]

    def __str__(self) -> str:
        return str(self._data)
    
    def __repr__(self) -> str:
        return str(self._data)

    def __getitem__(self, key) -> float:
        return self._data[key]

    
    @staticmethod
    def _element_wise_compatible(M1: Matrix, M2: Matrix) -> bool:
        if M1.shape != M2.shape or M1.size != M2.size:
            raise ValueError(f'Matrix of shape {M1.shape} is incompatible for method "{sys._getframe(1).f_code.co_name}" with matrix of shape {M2.shape}.')


    @property 
    def shape(self) -> tuple[int, int]:
        """Shape of matrix as (rows, columns)."""
        return self._shape
    
    @property
    def size(self) -> int:
        """Number of elements in matrix."""
        return self._size
    
    @property
    def T(self) -> Matrix:
        """Transpose of matrix."""


    @classmethod
    def new(cls, shape: tuple[int, int], fill = None) -> Matrix:
        """
        Returns an empty matrix of `shape` passed. Can be filled with a 
        filler value passed.

        Parameters
        ----------
        `shape` : `tuple[int, int]`
            Shape of new empty matrix.
        `filler` : `Any`, `default=None`
            Value to place in all elements.

        Returns
        -------
        `Matrix`
            Empty matrix.
        """

        with ThreadPool(cls._cpu_count) as pool:
            arr = pool.starmap(_build_arr, [(shape[1], fill) for _ in range(shape[0])])

        return arr


    def add(self, M: Matrix) -> Matrix:
        """
        Adds 2 matrices together element-wise

        Parameters
        ----------
        `M` : `Matrix`
            Matrix values to add to this matrix's values.

        Returns
        -------
        `Matrix`
            Result of matrix addition
        """

        # check compatibility
        self._element_wise_compatible(self, M)

        # multiprocessing to add rows at a time
        for _ in range(self.shape[0]):
            with ThreadPool(self._cpu_count) as pool:
                result = pool.starmap(_add_arr, [(self[i], M[i]) for i in range(self.shape[1])])

        return result
        
    def reshape(self, shape: tuple[int, int]) -> Matrix:
        """
        Reshapes matrix into new `shape` passed.

        Parameters
        ----------
        `shape` : `tuple[int, int]`
            Shape of resulting matrix

        Returns
        -------
        `Matrix`
            Reshaped matrix
        """

        # check compatibility
        if self.size != shape[0] * shape[1]:
            raise ValueError(f'{self.__class__.__name__} of shape {self.shape} with {self.size} element(s) cannot be reshaped into {shape} with {shape[0] * shape[1]} element(s).')

        # flatten first and get raw list
        flattened = self.flatten(reshape='flat')

        # funny list comprehension to reshape
        return Matrix([[flattened[(shape[1] * y) + x] for x in range(shape[1])] for y in range(shape[0])])

    def flatten(self, reshape: Literal['tall', 'long', 'flat'] = 'flat') -> list | Matrix:
        """
        Flattens the matrix into shape (1, N) if 'flat' is passed or (N, 1) if 
        'tall' is passed. 

        Parameters
        ----------
        `reshape` : `Literal['tall', 'long', 'flat']`, `default='flat'`
            Controls return shape of flattened matrix

        Returns
        -------
        `Matrix`
            Flattened matrix
        """

        # flatten into flat list
        flattened = [self._data[y][x] for y in range(self._shape[0]) for x in range(self._shape[1])]

        if reshape == 'flat':
            return flattened

        if reshape == 'long':
            return Matrix([flattened])
        
        return Matrix([[v] for v in flattened])
        

16


In [5]:
# %%timeit
Matrix([[1, 2], [3, 4]]).add(Matrix([[5, 6], [7, 8]]))

[[6, 8], [10, 12]]

In [32]:
import numpy as np

a = time.perf_counter()
np.zeros((16, 100000000))
print(time.perf_counter() - a)

a = time.perf_counter()
Matrix.new((16, 100000000))
print(time.perf_counter() - a)

0.07669269999996686
11.15724339999997


0.07304580000004535


[[None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,


In [17]:
def _add_arr(arr1, arr2) -> list[float]:
    return [v + arr2[i] for i, v in enumerate(arr1)]

A = [[1, 2], [3, 4]]
B = [[5, 6], [7, 8]]
C = []




In [18]:
%%timeit
for i, v in enumerate(A):
    C.append(_add_arr(v, B[i]))

1.37 µs ± 16.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [5]:
M = Matrix([[1, 2, 3],
            [4, 5, 6],
            [7, 8, 9],
            [10, 11, 12],
            [13, 14, 15],
            [16, 17, 18]])

In [6]:
M._add_arr([1], [2])

[3]

In [198]:
M.add(Matrix([[1, 2]]))

ValueError: Matrix of shape (6, 3) is incompatible for method "add" with matrix of shape (1, 2).

In [187]:
M.reshape((2, 9))
M.reshape((9, 2))

[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11, 12], [13, 14], [15, 16], [17, 18]]

In [166]:
M.reshape((9, 2))


[[1, 2, 3, 4, 5, 6, 7, 8, 9], [4, 5, 6, 7, 8, 9, 10, 11, 12]]

In [119]:
M.flatten(reshape='tall')

[[1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15]]

In [21]:
import math
math.sqrt(-1)

ValueError: math domain error

In [24]:
(3j).conjugate()

-3j

In [27]:
0j + 1j * 1j

(-1+0j)

In [ ]:
(1)(sqrt(-1)) * (1)(sqrt(-1))


In [29]:
math.sqrt(2) * math.sqrt(2)

2.0000000000000004